In [2]:
import numpy as np
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from numpy import linalg as LA
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import PCA
from scipy.sparse import coo_matrix
import pickle
import pandas as pd
from random import randint

import kmapper as km
from sklearn.manifold import TSNE

def convertNansToZeros(ma):
    nan_elements = np.flatnonzero(np.isnan(ma.data))
    if len(nan_elements) > 0:
        ma.data[nan_elements] = 0
    return ma


def convertInfsToZeros(ma):
    inf_elements = np.flatnonzero(np.isinf(ma.data))
    if len(inf_elements) > 0:
        ma.data[inf_elements] = 0
    return ma



In [8]:
%matplotlib
datadir = '/home/garner1/Work/dataset/SSF/prostate-twelve/'
section = '1.1'
filename = datadir+'P'+section+'.tsv'

df = pd.read_csv(filename, sep='\t', header=0) # read the gene X position count-matrix

spatialCoord = [w.split('x') for w in list(df.columns.values)]
spatialCoord = [map(int,pairs) for pairs in spatialCoord]
i = [l[0] for l in spatialCoord]
j = [l[1] for l in spatialCoord]

dfnorm = df.div(df.sum(axis=0),axis=1) #normalize to probabilities the transcriptome at each position

# data = dfnorm.values.transpose() # n_spots X n_genes
data = dfnorm.values
# data = df.values

# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projection = TSNE(n_components=3,perplexity=50)
projected_data = mapper.fit_transform(data, projection=projection) # X-Y axis
nr_cubes = 10
# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, data, nr_cubes = nr_cubes)

# Visualize it
mapper.visualize(graph, path_html=section+".TSNE_mapper_output.html",title=filename)

Using matplotlib backend: Qt5Agg
..Composing projection pipeline length 1:
Projections: TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=3, n_iter=1000, n_iter_without_progress=300, perplexity=50,
   random_state=None, verbose=0)


Distance matrices: False


Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))


..Projecting on data shaped (16209, 432)

..Projecting data using: 
	TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=3, n_iter=1000, n_iter_without_progress=300, perplexity=50,
   random_state=None, verbose=1)

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 16209 samples in 0.912s...
[t-SNE] Computed neighbors for 16209 samples in 190.208s...
[t-SNE] Computed conditional probabilities for sample 1000 / 16209
[t-SNE] Computed conditional probabilitie

/home/garner1/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


u'<!DOCTYPE html>\n<html>\n\n<head>\n  <div id="json_graph" data-graph=\'{"nodes": [{"name": "cube503_cluster0", "color": 10, "tooltip": "<h3>Size</h3><p>19</p>", "type": "circle", "id": "", "size": 3}, {"name": "cube427_cluster0", "color": 17, "tooltip": "<h3>Size</h3><p>47</p>", "type": "circle", "id": "", "size": 4}, {"name": "cube569_cluster0", "color": 16, "tooltip": "<h3>Size</h3><p>24</p>", "type": "circle", "id": "", "size": 4}, {"name": "cube646_cluster0", "color": 23, "tooltip": "<h3>Size</h3><p>72</p>", "type": "circle", "id": "", "size": 5}, {"name": "cube278_cluster0", "color": 7, "tooltip": "<h3>Size</h3><p>3</p>", "type": "circle", "id": "", "size": 2}, {"name": "cube785_cluster0", "color": 12, "tooltip": "<h3>Size</h3><p>12</p>", "type": "circle", "id": "", "size": 3}, {"name": "cube651_cluster0", "color": 13, "tooltip": "<h3>Size</h3><p>43</p>", "type": "circle", "id": "", "size": 4}, {"name": "cube656_cluster0", "color": 20, "tooltip": "<h3>Size</h3><p>84</p>", "type"

In [ ]:
gene = randint(0,df.shape[1])
data = dfnorm.values.tolist()[gene]
mat = coo_matrix((data, (i, j)), [max(i)+1, max(j)+1]).todense()

cmap = sns.cubehelix_palette(light=1, as_cmap=True)
fig = sns.heatmap(mat,annot=False,cmap=cmap)
fig.set_title(list(df.index)[gene])